In [3]:
import geopandas as gpd
from rasterstats import zonal_stats
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_bounds
import xarray as xr
import os

admin_vector_path = '臺北市區界圖_20220915/G97_A_CADIST_P.shp'
stat_min_vector_path = 'STAT/113年12月臺北市統計區人口指標_最小統計區/113年12月臺北市統計區人口指標_最小統計區_SHP/113年12月臺北市統計區人口指標_最小統計區.SHP'
output_dir = 'output'

date_mod11a2 = '2022257'
date_mod13q1 = '2022257'
mod11a2_nc_path = f'output/MOD11A2/MOD11A2_A{date_mod11a2}_WGS84.nc'
mod13q1_nc_path = f'output/MOD13Q1/MOD13Q1_A{date_mod13q1}_WGS84.nc'

def compute_zonal_statistics_modis(vector_path, nc_path, output_geojson='result.geojson', variable_name='LST', convert_to_celsius=False):
    """
    計算 NetCDF (MODIS) 與 vector 的 zonal statistics
    支援 MOD11A2 (LST) 和 MOD13Q1 (NDVI) 產品

    Parameters:
    -----------
    vector_path : str
        Vector shapefile 或 GeoJSON 檔案路徑
    nc_path : str
        NetCDF 檔案路徑 (MODIS)
    output_geojson : str
        輸出 GeoJSON 檔案路徑（預設為 'result.geojson'）
    variable_name : str
        NetCDF 變數名稱（'LST' for MOD11A2, 'NDVI' for MOD13Q1）
    convert_to_celsius : bool
        是否將 Kelvin 轉換為 Celsius（僅適用於 LST，預設為 False）

    Returns:
    --------
    geopandas.GeoDataFrame
        包含 zonal statistics 結果的 GeoDataFrame
    """

    # 1. 讀取 vector layer 並轉換到 WGS84
    vector = gpd.read_file(vector_path)

    print(f"Vector 原始 CRS: {vector.crs}")
    if vector.crs is None:
        print("設定 Vector CRS 為 TWD97 (EPSG:3826)")
        vector.crs = 'EPSG:3826'

    if vector.crs != 'EPSG:4326':
        print(f"Vector CRS: {vector.crs} -> 轉換到 WGS84 (EPSG:4326)")
        vector = vector.to_crs('EPSG:4326')

    # 2. 讀取 NetCDF 並轉換為臨時 GeoTIFF
    print(f"讀取 NetCDF 檔案: {nc_path}")
    ds = xr.open_dataset(nc_path)
    
    # 取得資料
    data = ds[variable_name].values
    lat = ds['lat'].values
    lon = ds['lon'].values
    
    print(f"NetCDF 資料維度: {data.shape}")
    print(f"緯度範圍: {lat.min():.4f} ~ {lat.max():.4f}")
    print(f"經度範圍: {lon.min():.4f} ~ {lon.max():.4f}")
    
    # 轉換 Kelvin 到 Celsius (僅適用於 LST)
    if convert_to_celsius and variable_name == 'LST':
        print("將溫度從 Kelvin 轉換為 Celsius")
        data = data - 273.15
    
    # 將 NaN 轉換為 nodata (-9999)
    nodata_value = -9999
    data_filled = np.where(np.isnan(data), nodata_value, data)
    
    # 建立 affine transform
    transform = from_bounds(
        lon.min(), lat.min(), lon.max(), lat.max(),
        data.shape[1], data.shape[0]
    )
    
    # 建立臨時 GeoTIFF
    temp_tif_path = nc_path.replace('.nc', '_temp.tif')
    
    with rasterio.open(
        temp_tif_path,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data_filled.dtype,
        crs='EPSG:4326',
        transform=transform,
        nodata=nodata_value
    ) as dst:
        # 注意：需要上下翻轉資料，因為 NetCDF 的 lat 是從上到下
        dst.write(np.flipud(data_filled), 1)
    
    print(f"已建立臨時 GeoTIFF: {temp_tif_path}")
    
    # 3. 定義 90% quantile 函數
    def p90(x):
        if hasattr(x, 'compressed'):
            data = x.compressed()
        else:
            data = np.array(x, copy=True)

        if len(data) == 0:
            return np.nan

        return np.percentile(data, 90)

    # 4. 執行 zonal statistics
    stats = zonal_stats(
        vector,
        temp_tif_path,
        stats=['mean', 'min', 'max'],
        add_stats={'p90': p90},
        nodata=nodata_value
    )

    # 5. 將統計結果加入 vector layer
    vector['mean'] = [s['mean'] for s in stats]
    vector['min'] = [s['min'] for s in stats]
    vector['max'] = [s['max'] for s in stats]
    vector['p90'] = [s['p90'] for s in stats]

    # 6. 儲存為 GeoJSON
    vector.to_file(output_geojson, driver='GeoJSON')
    print(f"結果已儲存至: {output_geojson}")

    # 7. 顯示統計摘要
    print("\n統計結果摘要:")
    if variable_name == 'LST':
        unit = "Celsius" if convert_to_celsius else "Kelvin"
        print(f"溫度單位: {unit}")
    else:
        print(f"變數: {variable_name}")
    print(vector[['mean', 'min', 'max', 'p90']].describe())
    
    # 清理臨時檔案
    ds.close()
    if os.path.exists(temp_tif_path):
        os.remove(temp_tif_path)
        print(f"已刪除臨時檔案: {temp_tif_path}")

    return vector


# 使用範例 - MOD11A2 (LST)
print("=== 處理 MOD11A2 (LST) ===")
vector_lst = compute_zonal_statistics_modis(
    vector_path= admin_vector_path,
    nc_path=mod11a2_nc_path,
    output_geojson=f'{output_dir}/result_mod11a2_admin_{date_mod11a2}.geojson',
    variable_name='LST',
    convert_to_celsius=True
)
vector_lst = compute_zonal_statistics_modis(
    vector_path= stat_min_vector_path,
    nc_path=mod11a2_nc_path,
    output_geojson=f'{output_dir}/result_mod11a2_statmin_{date_mod11a2}.geojson',
    variable_name='LST',
    convert_to_celsius=True
)

# 使用範例 - MOD13Q1 (NDVI)
print("\n=== 處理 MOD13Q1 (NDVI) ===")
vector_ndvi = compute_zonal_statistics_modis(
    vector_path= admin_vector_path,
    nc_path=mod13q1_nc_path,
    output_geojson=f'{output_dir}/result_mod13q1_admin_{date_mod13q1}.geojson',
    variable_name='NDVI',
    convert_to_celsius=False
)
# 使用範例 - MOD13Q1 (NDVI)
print("\n=== 處理 MOD13Q1 (NDVI) ===")
vector_ndvi = compute_zonal_statistics_modis(
    vector_path= stat_min_vector_path,
    nc_path=mod13q1_nc_path,
    output_geojson=f'{output_dir}/result_mod13q1_statmin_{date_mod13q1}.geojson',
    variable_name='NDVI',
    convert_to_celsius=False
)

=== 處理 MOD11A2 (LST) ===
Vector 原始 CRS: EPSG:3826
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
讀取 NetCDF 檔案: output/MOD11A2/MOD11A2_A2022257_WGS84.nc
NetCDF 資料維度: (1698, 5460)
緯度範圍: 20.0053 ~ 29.9971
經度範圍: 106.4207 ~ 138.5627
將溫度從 Kelvin 轉換為 Celsius
已建立臨時 GeoTIFF: output/MOD11A2/MOD11A2_A2022257_WGS84_temp.tif
結果已儲存至: output/result_mod11a2_admin_2022257.geojson

統計結果摘要:
溫度單位: Celsius
            mean        min        max        p90
count  12.000000  12.000000  12.000000  12.000000
mean   28.341338  25.968119  31.046435  30.075096
std     3.002806   2.429107   3.475634   3.465101
min    24.288018  21.844360  26.120697  25.417938
25%    25.519402  24.675285  28.706703  26.589719
50%    28.046072  26.164307  30.369614  29.334970
75%    30.182516  27.062172  34.420059  33.078050
max    33.923625  29.807343  35.566864  35.041946
已刪除臨時檔案: output/MOD11A2/MOD11A2_A2022257_WGS84_temp.tif
Vector 原始 CRS: None
設定 Vector CRS 為 TWD97 (EPSG:3826)
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
讀取 N

In [6]:
import geopandas as gpd
from rasterstats import zonal_stats
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

def compute_zonal_statistics(vector_path, tif_path, output_geojson='result.geojson', nodata_value=-1):
    """
    計算 raster 與 vector 的 zonal statistics

    Parameters:
    -----------
    vector_path : str
        Vector shapefile 或 GeoJSON 檔案路徑
    tif_path : str
        GeoTIFF 檔案路徑
    output_geojson : str
        輸出 GeoJSON 檔案路徑（預設為 'result.geojson'）
    nodata_value : int or float
        要視為 NoData 的值（預設為 -1）

    Returns:
    --------
    geopandas.GeoDataFrame
        包含 zonal statistics 結果的 GeoDataFrame
    """

    # 1. 讀取 vector layer 並轉換到 WGS84
    vector = gpd.read_file(vector_path)

    print(f"Vector 原始 CRS: {vector.crs}")
    if vector.crs is None:
        print("設定 Vector CRS 為 TWD97 (EPSG:3826)")
        vector.crs = 'EPSG:3826'

    if vector.crs != 'EPSG:4326':
        print(f"Vector CRS: {vector.crs} -> 轉換到 WGS84 (EPSG:4326)")
        vector = vector.to_crs('EPSG:4326')

    # 2. 將 TIF 檔重新投影到 WGS84，並設定 nodata
    tif_wgs84_path = tif_path.replace('.tif', '_wgs84.tif')

    with rasterio.open(tif_path) as src:
        print(f"TIF 檔 CRS: {src.crs}")

        if src.crs != 'EPSG:4326':
            print(f"TIF 檔需要從 {src.crs} 轉換到 WGS84")

            transform, width, height = calculate_default_transform(
                src.crs, 'EPSG:4326', src.width, src.height, *src.bounds
            )

            kwargs = src.meta.copy()
            kwargs.update({
                'crs': 'EPSG:4326',
                'transform': transform,
                'width': width,
                'height': height,
                'nodata': nodata_value  # 設定 nodata 值
            })

            with rasterio.open(tif_wgs84_path, 'w', **kwargs) as dst:
                reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs='EPSG:4326',
                    resampling=Resampling.bilinear,
                    dst_nodata=nodata_value  # 設定目標 nodata 值
                )

            print(f"TIF 檔已重新投影到 WGS84: {tif_wgs84_path}")
        else:
            tif_wgs84_path = tif_path

    # 3. 定義 90% quantile 函數
    def p90(x):
        if hasattr(x, 'compressed'):
            data = x.compressed()
        else:
            data = np.array(x, copy=True)

        if len(data) == 0:
            return np.nan

        return np.percentile(data, 90)

    # 4. 執行 zonal statistics（nodata 值會自動被忽略）
    stats = zonal_stats(
        vector,
        tif_wgs84_path,
        stats=['mean', 'min', 'max'],
        add_stats={'p90': p90},
        nodata=nodata_value  # 明確指定 nodata 值
    )

    # 5. 將統計結果加入 vector layer
    vector['mean'] = [s['mean'] for s in stats]
    vector['min'] = [s['min'] for s in stats]
    vector['max'] = [s['max'] for s in stats]
    vector['p90'] = [s['p90'] for s in stats]

    # 6. 儲存為 GeoJSON
    vector.to_file(output_geojson, driver='GeoJSON')
    print(f"結果已儲存至: {output_geojson}")

    # 7. 顯示統計摘要
    print("\n統計結果摘要:")
    print(vector[['mean', 'min', 'max', 'p90']].describe())

    return vector

date = '2022251'
# 使用範例
vector = compute_zonal_statistics(
    vector_path= stat_min_vector_path,
    tif_path=f'ndvi_output/lst_result_fixed_{date}.tif',
    output_geojson=f'{output_dir}/result_lst_minstatic_{date}.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)

vector = compute_zonal_statistics(
    vector_path= admin_vector_path,
    tif_path=f'ndvi_output/lst_result_fixed_{date}.tif',
    output_geojson=f'{output_dir}/result_lst_admin_{date}.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)



vector = compute_zonal_statistics(
    vector_path=stat_min_vector_path,
    tif_path=f'ndvi_output/ndvi_result_fixed_{date}.tif',
    output_geojson=f'{output_dir}/result_ndvi_minstatic_{date}.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)

vector = compute_zonal_statistics(
    vector_path=admin_vector_path,
    tif_path=f'ndvi_output/ndvi_result_fixed_{date}.tif',
    output_geojson=f'{output_dir}/result_ndvi_admin_{date}.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)


Vector 原始 CRS: None
設定 Vector CRS 為 TWD97 (EPSG:3826)
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
TIF 檔 CRS: EPSG:32651
TIF 檔需要從 EPSG:32651 轉換到 WGS84
TIF 檔已重新投影到 WGS84: ndvi_output/lst_result_fixed_2022251_wgs84.tif
結果已儲存至: output/result_lst_minstatic_2022251.geojson

統計結果摘要:
               mean           min           max           p90
count  11418.000000  11418.000000  11418.000000  11418.000000
mean      30.285794     29.803364     30.778362     30.634265
std        1.802850      2.229603      1.556975      1.649378
min        2.740272     -0.919702      2.740272      2.740272
25%       29.550667     29.082102     30.042207     29.895520
50%       30.667473     30.319167     31.033751     30.947824
75%       31.456633     31.170840     31.787075     31.698559
max       34.141107     34.015858     38.318753     36.695094
Vector 原始 CRS: EPSG:3826
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
TIF 檔 CRS: EPSG:32651
TIF 檔需要從 EPSG:32651 轉換到 WGS84
TIF 檔已重新投影到 WGS84: ndvi_output/lst_res

In [32]:
vector = compute_zonal_statistics(
    # vector_path='STAT/113年12月臺北市統計區人口指標_最小統計區/113年12月臺北市統計區人口指標_最小統計區_SHP/113年12月臺北市統計區人口指標_最小統計區.SHP',
    vector_path='臺北市區界圖_20220915/G97_A_CADIST_P.shp',
    tif_path='ndvi_output/lst_result_fixed.tif',
    output_geojson='result_lst_admin.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)
vector = compute_zonal_statistics(
    # vector_path='STAT/113年12月臺北市統計區人口指標_最小統計區/113年12月臺北市統計區人口指標_最小統計區_SHP/113年12月臺北市統計區人口指標_最小統計區.SHP',
    vector_path='臺北市區界圖_20220915/G97_A_CADIST_P.shp',
    tif_path='ndvi_output/ndvi_result_fixed_2024185.tif',
    output_geojson='result_ndvi_admin.geojson',
    nodata_value=-1  # 將 -1 視為 NoData
)

Vector 原始 CRS: EPSG:3826
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
TIF 檔 CRS: EPSG:32651
TIF 檔需要從 EPSG:32651 轉換到 WGS84
TIF 檔已重新投影到 WGS84: ndvi_output/lst_result_fixed_wgs84.tif
結果已儲存至: result_lst_admin.geojson

統計結果摘要:
            mean        min        max        p90
count  12.000000  12.000000  12.000000  12.000000
mean   28.585955  16.353177  34.376926  31.788481
std     2.686231  12.771866   0.658397   1.179564
min    23.536089  -0.986408  33.433716  30.342674
25%    26.832615  -0.861843  34.016105  30.594800
50%    29.608114  24.557453  34.370203  31.909385
75%    30.652287  25.327858  34.544081  32.633678
max    31.474989  25.998840  35.780739  33.561933
Vector 原始 CRS: EPSG:3826
Vector CRS: EPSG:3826 -> 轉換到 WGS84 (EPSG:4326)
TIF 檔 CRS: EPSG:32651
TIF 檔需要從 EPSG:32651 轉換到 WGS84
TIF 檔已重新投影到 WGS84: ndvi_output/ndvi_result_fixed_2024185_wgs84.tif
結果已儲存至: result_ndvi_admin.geojson

統計結果摘要:
            mean        min        max        p90
count  12.000000  12.000000  12.000000  12